In [1]:
import dask.dataframe as dd
import datetime
from collections import defaultdict
import numpy as np
import time
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from math import ceil
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

#hierarchical clustering
from sklearn.cluster import AgglomerativeClustering, KMeans
from scipy.cluster.hierarchy import dendrogram, linkage, set_link_color_palette
#Kmeans
from sklearn.cluster import KMeans

In [21]:
df = pd.read_csv('df_cat_quarter.csv')
df.drop(columns = 'Unnamed: 0', inplace = True)
df.columns = ['store', 'Category', 'year', 'quarter', 'u', 'v']

In [22]:
#calculate average price
table = pd.pivot_table(df, values=['v','u'], index=['Category'], aggfunc=([np.sum]))
table.columns = table.columns.droplevel(0)
table['avg_price'] = table['v']/table['u']
df_price = table

In [23]:
table

,u,v,avg_price
Category,,,
1,1234152.0,9.514109e+08,770.902559
2,4198685.5,2.946828e+09,701.845394
3,4667917.5,3.840373e+09,822.716530
4,287340.0,4.382875e+08,1525.327264
5,40739.0,1.770351e+07,434.559268
...,...,...,...
174,647.0,5.012130e+05,774.672334
175,11031314.0,1.997629e+10,1810.871008
176,9.0,2.650700e+04,2945.222222


In [24]:
table = pd.pivot_table(df, values='u', index=['Category'],
                    columns=['quarter'], aggfunc=([np.sum]))
#average per year per month in the quarter. Quarter 4 has 1 month only for 2019, hence 3+3+1
table['q1'] = table[('sum', 1)]/9
table['q2'] = table[('sum', 2)]/9
table['q3'] = table[('sum', 3)]/9
table['q4'] = table[('sum', 4)]/7
table[['q1','q2','q3','q4']] = table[['q1','q2','q3','q4']].round(0)
table.drop(columns = [('sum',  1),('sum',  2),('sum',  3),('sum',  4)], inplace = True)

table = table.reset_index()
table.columns = table.columns.droplevel(1)
table.set_index('Category', inplace = True)
table = table.fillna(0)
quarter_cat = table
res = quarter_cat.div(quarter_cat.sum(axis=1), axis=0)
res = res.fillna(0)

In [25]:
units_cat.columns

MultiIndex([('sum', 2016),
            ('sum', 2017),
            ('sum', 2018),
            ('sum', 2019)],
           names=[None, 'year'])

In [26]:
units_cat = pd.pivot_table(df, values='u', index=['Category'],
                    columns=['year'], aggfunc=([np.sum]))
units_cat = units_cat.fillna(0)
units_cat = units_cat.reset_index()
units_cat.set_index('Category', inplace = True)
units_cat[('sum', 2018)] = units_cat[('sum', 2018)]/12*10

In [27]:
units_cat

sum                                      
year            2016        2017          2018        2019
Category                                                  
1           326295.0    297209.0  2.882925e+05    264697.0
2          1118289.0   1041238.5  9.484833e+05    900978.0
3          1430571.5   1151875.0  1.003085e+06    881768.5
4            71598.0     70784.0  6.445417e+04     67613.0
5            11623.0     10978.0  8.854167e+03      7513.0
...              ...         ...           ...         ...
174            167.0       138.0  1.366667e+02       178.0
175        3021874.0   2917410.0  2.344648e+06   2278452.0
176              4.0         5.0  0.000000e+00         0.0
177           2426.0      1538.0  9.041667e+02      1080.0
178       34313435.5  35274373.5  3.081822e+07  32780109.5

[178 rows x 4 columns]

In [28]:
units_cat.columns = units_cat.columns.droplevel(0)
units_cat['v%vsLY'] = units_cat[2019]/units_cat[2018]-1
units_cat['v%vsLLY'] = units_cat[2018]/units_cat[2017]-1
units_cat.drop(columns=2017, inplace =True)

In [32]:
store_count = pd.pivot_table(df, values='store', columns ='year', index=['Category'],
                    aggfunc=([pd.Series.nunique]))

In [33]:
store_count = store_count.fillna(0)
store_count = store_count.reset_index()
store_count.set_index('Category', inplace = True)
store_count.columns = store_count.columns.droplevel(0)

In [34]:
store_count['v%vsLY'] = store_count[2019]/store_count[2018]-1
store_count['v%vsLLY'] = store_count[2018]/store_count[2017]-1

In [35]:
store_count = store_count.fillna(0)

In [36]:
store_count.drop(columns=[2016,2017,2018], inplace =True)

In [37]:
units_cat.drop(2016, axis = 1, inplace = True)

In [38]:
df_c = units_cat.merge(store_count, right_index = True, left_index = True)

In [39]:
df_c

year,2018,2019_x,v%vsLY_x,v%vsLLY_x,2019_y,v%vsLY_y,v%vsLLY_y
Category,,,,,,,
1,2.882925e+05,264697.0,-0.081846,-0.030001,410.0,0.000000,0.000000
2,9.484833e+05,900978.0,-0.050086,-0.089082,410.0,0.000000,0.000000
3,1.003085e+06,881768.5,-0.120944,-0.129172,410.0,0.000000,0.000000
4,6.445417e+04,67613.0,0.049009,-0.089425,410.0,0.000000,0.000000
5,8.854167e+03,7513.0,-0.151473,-0.193463,356.0,-0.021978,-0.010870
...,...,...,...,...,...,...,...
174,1.366667e+02,178.0,0.302439,-0.009662,29.0,0.160000,0.086957
175,2.344648e+06,2278452.0,-0.028233,-0.196325,410.0,0.000000,0.000000
176,0.000000e+00,0.0,NaN,-1.000000,0.0,0.000000,-1.000000


In [40]:
df_c = df_c.merge(res, right_index = True, left_index = True )

In [41]:
df_c.columns = [ 'u_18',    'u_19',  'units%vsLY', 'units%vsLLY',    'store_count_19',
        'count%vsLY', 'count%vsLLY', 'q1','q2','q3','q4']

In [42]:
number_stores = 410
df_c['store_count_19'] = df_c['store_count_19']/number_stores

In [43]:
df_c.to_csv('df_cat_clustering_3.csv')